<a href="https://colab.research.google.com/github/c-c-c-c/dm_integration/blob/master/myMecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストの分類
テキスト分類は、以下の順に行います。

1. 前処理
2. 数値表現化 (CountVectorizer, TfIdfVectorizer)
3. 分類器を学習

本ノートブックでは、Wikipedia のエントリ分類を通じて、テキスト分類器を作成します。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+mai

In [3]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [0]:
import joblib
import MeCab
import numpy as np
import pandas as pd
import re

from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC

## データの読み込み

In [0]:
# EPGデータ(手作業での修正中))
df = pd.read_excel("./drive/My Drive/0_インテグ作業/data/EPG_checking0212.xlsx")

In [0]:
df_cat = 



In [0]:
# 空の列を足す
df["sharp_epg_tknz"] = np.nan

In [0]:
# ゴミ除去

def removeTrash (text):
    import re

    result_text = text
    result_text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", result_text)
    result_text = re.sub(r"番組詳細|制作・著作|制作著作", "", result_text)
    result_text = re.sub(r"[!\(\)=『』～/]", "", result_text)
    result_text = re.sub(r"フジテレビ|日本テレビ|TBS|テレビ朝日|TBS|関西テレビ", "", result_text)
    
    result_text = re.sub(r"【公式.*?】", "", result_text)
    result_text = re.sub(r"\u3000", "", result_text)

    return result_text


In [57]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df['sharp_epg_hand_corrected']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df["sharp_epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# 手作業での前処理が完了してないので、完了したものだけにする

bool_list = [] 

for i in range(len(df["sharp_epg_hand_corrected"])):
    bool_list.append( type( df["sharp_epg_tknz"].iloc[i] ) != float )

In [0]:
df_notnull =  df[bool_list]

In [0]:
# df = pd.DataFrame( columns=['A','B'] )

# # 行を作る
# tmp_se = pd.Series( [ i, i*i ], index=list_df.columns )

# # dfに行をレコードとして追加
# df = df.append( tmp_se, ignore_index=True )


In [48]:

# print(df["sharp_epg_hand_corrected"].iloc[3110])

pandas.core.series.Series

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:

X = vectorizer.fit_transform(
    [str(i) for i in df["sharp_epg_tknz"].values]
)

In [18]:
X

<6154x23218 sparse matrix of type '<class 'numpy.float64'>'
	with 307732 stored elements in Compressed Sparse Row format>

In [0]:
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

In [0]:
vectorizer.idf_

In [0]:
# idf値の確認

idf = pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
    .to_frame("idf") \
    .sort_values("idf", ascending=False)

In [0]:


pd.set_option('display.max_rows', 1000)
idf.iloc[1:100]

## 形態素解析
---

MeCab を利用した形態素解析と、簡単な処理の例を示します。

### 簡単な例

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

CountVectorizer が生成する単語文書行列では、行 (横) 方向が1つの文書を表し、それぞれの列に単語の出現回数が格納されます (※単語文書行列と言った場合、列方向が文書を表す場合もあるので、他の文献を読む際は注意が必要です)。

どの列がどの単語と対応しているかを知るためには、vectorizer の vocabulary_ を参照します。

In [0]:
vectorizer.vocabulary_

新しい文書に対しては、transform で変換します。

なお、fit\_transform の際に出てこなかった単語 (= vocabulary_ にない単語) については、行列内に登場しません。

In [0]:
# transform も、リスト形式で与える必要があるので注意
X_new = vectorizer.transform([
    "Hello I like this pen"
])
X_new.toarray()

### 日本語データに適用
前処理済みの日本語データに適用してみます。DataFrame にスペース区切りのテキストを準備している場合、DataFrame の列を参照させたものをそのまま与えることができます。

In [0]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X

In [0]:
X[0].toarray()

↑この規模の単語文書行列になると、行列のサイズ (文書数x単語数) と比較し、値が入っている場所をほとんど無いことが分かります。

In [0]:
vectorizer.vocabulary_

## TF-IDF変換
---

TF-IDF 変換を行うと、単語の出現回数に、他の文書全体と比較した際の希少性 (レア度) で重みを付けることができます。

TF-IDF 変換済みの単語文書行列は、`TfidfVectorizer` で作成できます。

### 簡単な例

In [0]:
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:
df[""]

In [0]:
# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])
X.toarray()

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

他の文書でも出現している単語については、同じ出現回数でも重みが低くなっていることが分かります。各単語の重みは、`idf_` で見ることができます。確認してみましょう。

In [0]:
vectorizer.idf_

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(np.atleast_2d(vectorizer.idf_), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

デフォルトでは、各文書のノルムが1になるように正規化されています (norm="l2")。確認してみましょう。

In [0]:
np.linalg.norm(X.toarray(), axis=1)

### 日本語データに適用
CountVectorizer の場合と同様に、前処理済みの日本語データに適用してみます。

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X[0].toarray()

IDF 値の大きい単語・小さい単語を確認してみましょう。

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
    .to_frame("idf") \
    .sort_values("idf", ascending=False)

## テキスト分類
---
定量化したテキスト情報を使うことで、テキストをカテゴリー別に分類してみましょう

In [0]:
# テストデータ
df_test = pd.read_csv("dataset/wikipedia-test.txt", sep="\t")
df_test.head()

### 単語文書行列 (TF-IDF 適用済み) の作成
単語文章行列を作成する前に、まずは文章を形態素解析します。

In [0]:
mecab = MeCab.Tagger("-O wakati")

text_tokenized = []
for text in df_test['text']:
    text_tokenized.append(mecab.parse(text))
    
df_test['text_tokenized'] = text_tokenized

続いて、単語文章行列を学習データテストデータでそれぞれ生成します。<br>
この時、2つの単語文章行列は同じ次元数でなければならないので、`transform`を使用します

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X_test = vectorizer.transform(df_test["text_tokenized"])

### 分類器の適用・比較
単語文書行列ができたら、後はこれまでやってきた機械学習と同様です。

いくつかの分類器で、性能を比較してみましょう。

In [0]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### ロジスティック回帰

In [0]:
clf_lr = LogisticRegression(n_jobs=-1)
clf_lr.fit(X, df["category"])

推定結果は predict で得ることができます (他の分類器でも同様)。

In [0]:
clf_lr.predict(X_test)

In [0]:
y_test_pred = clf_lr.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

### SVM (線形カーネル)

In [0]:
clf_svc = LinearSVC()
clf_svc.fit(X, df["category"])

In [0]:
y_test_pred = clf_svc.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

### Random Forest

In [0]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(X, df["category"])

In [0]:
y_test_pred = clf_rf.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

## 【演習1】
上記いずれかのアルゴリズムのパラメータを変更して、分類精度を高めてみましょう。

In [0]:
# Enter your code here

### モデルの保存
`pickle` でも保存できますが、`joblib` を使い、ファイル名末尾に `.gz`, `.bz2` 等を指定すると、自動的に圧縮してくれます。読み込み時も同様、自動的に解凍して読み込んでくれます。

言語処理ではモデルが大容量になることが多いため、モデルの圧縮は重要です。

In [0]:
joblib.dump(clf_rf, "wikipedia_category_classifier.pkl.gz")

### モデル読み込みテスト

In [0]:
clf_rf_restored = joblib.load("wikipedia_category_classifier.pkl.gz")
clf_rf_restored

読み込んだモデルは、上記ですでに学習済みなので、`predict`で予測が行えます

In [0]:
clf_rf_restored.predict(X_test)